In [17]:
import pandas as pd
import os

These are the datasets available:

In [18]:
print(os.listdir('flow data/flow data'))

['usgs_san_francisquito.txt', 'usgs_data.csv', 'influent_flow.csv', 'weather.csv']


The usgs_data.csv is just the table from usgs_san_francisquito.txt

Let's take a look at each dataset and see if any cleaning/manipulation needs to be done

In [19]:
#Looking at influent_flow.csv table, our target
flow = pd.read_csv('flow data/flow data/influent_flow.csv')
print('Rows: ', flow.shape[0], '\nColumns: ', flow.shape[1])
print(flow.info())

#Change the date column to a datetime for this table as well
flow['Date'] = pd.to_datetime(flow['Date'])

flow.head()

Rows:  34516 
Columns:  2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34516 entries, 0 to 34515
Data columns (total 2 columns):
Date        34516 non-null object
INF_FLOW    34516 non-null float64
dtypes: float64(1), object(1)
memory usage: 539.4+ KB
None


,Date,INF_FLOW
0,2013-01-15 00:00:00,9.589273
1,2013-01-15 01:00:00,9.272833
2,2013-01-15 02:00:00,8.861329
3,2013-01-15 03:00:00,9.193454
4,2013-01-15 04:00:00,9.129482


In [20]:
#Looking at weather table
weather = pd.read_csv('flow data/flow data/weather.csv')
print('Rows: ', weather.shape[0], '\nColumns: ', weather.shape[1])
print(weather.info())

#Change the date column to a datetime data type for easier manipulation
weather['Date'] = pd.to_datetime(weather['Date'])

weather.head()

Rows:  44784 
Columns:  8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44784 entries, 0 to 44783
Data columns (total 8 columns):
Date               44784 non-null object
rain               44784 non-null float64
snow               44784 non-null float64
accum              44784 non-null float64
temp               44784 non-null float64
humidity           44784 non-null float64
cloud              44784 non-null float64
app_temperature    44784 non-null float64
dtypes: float64(7), object(1)
memory usage: 2.7+ MB
None


,Date,rain,snow,accum,temp,humidity,cloud,app_temperature
0,2012-12-15 08:00:00,0.0,0.0,0.0,5.95,0.89,0.08,5.95
1,2012-12-15 09:00:00,0.0,0.0,0.0,5.66,0.92,0.09,5.66
2,2012-12-15 10:00:00,0.0,0.0,0.0,5.46,0.91,0.09,5.46
3,2012-12-15 11:00:00,0.0,0.0,0.0,4.82,0.92,0.06,4.82
4,2012-12-15 12:00:00,0.0,0.0,0.0,4.61,0.93,0.03,4.61


In [22]:
#Looking at usgs_san_francisuito.txt table
usgs = pd.read_csv('flow data/flow data/usgs_data.csv', sep = '\t', low_memory = False)
print('Rows: ', usgs.shape[0], '\nColumns: ',usgs.shape[1])
print(usgs.info()) #Looks like we have some missing values in the last 4 columns

#get rid of row 0 (indicating datatype and length for each column)
usgs.drop(0, inplace= True)

#Change the date column to a datetime for this table as well
usgs['datetime'] = pd.to_datetime(usgs['datetime'])

#The original usgs text file indicates the 14747_00060 and 14748_0065 columns 
#indicate discharge and gage height. Lets change the name of those columns
#and their corresponding qualification codes. Let's also rename the 'datetime'
#to 'Date' to be consitent with the other tables

usgs.rename(columns = {'14747_00060': 'discharge', 
            '14747_00060_cd': 'discharge_cd',
            '14748_00065' : 'gage_height',
            '14748_00065_cd' : 'gage_height_cd',
            'datetime' : 'Date'},
           inplace = True)

usgs.head()

Rows:  178896 
Columns:  8
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178896 entries, 0 to 178895
Data columns (total 8 columns):
agency_cd         178896 non-null object
site_no           178896 non-null object
datetime          178896 non-null object
tz_cd             178896 non-null object
14747_00060       178676 non-null object
14747_00060_cd    178676 non-null object
14748_00065       178761 non-null object
14748_00065_cd    178761 non-null object
dtypes: object(8)
memory usage: 10.9+ MB
None


,agency_cd,site_no,Date,tz_cd,discharge,discharge_cd,gage_height,gage_height_cd
1,USGS,11164500,2013-01-01 00:00:00,PST,44.1,A,1.27,A
2,USGS,11164500,2013-01-01 00:15:00,PST,44.1,A,1.27,A
3,USGS,11164500,2013-01-01 00:30:00,PST,44.1,A,1.27,A
4,USGS,11164500,2013-01-01 00:45:00,PST,43.2,A,1.26,A
5,USGS,11164500,2013-01-01 01:00:00,PST,43.2,A,1.26,A


Now that the 3 dataframes have been cleaned up a bit lets merge them together by date